In [1]:
import os
os.chdir(r"C:\Users\User\Desktop\Data_Science_Program\CNN_PROJECTS\CodeBase\deppCNNClassifier")
print(os.getcwd())

C:\Users\User\Desktop\Data_Science_Program\CNN_PROJECTS\CodeBase\deppCNNClassifier


In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallBacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [3]:
from deppClassifier.constants import * 
from deppClassifier.utils import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallBacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config = PrepareCallBacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
           
        )

        return prepare_callbacks_config

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallBacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at{timestamp}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only = True
        )


    def get_tb_ckpt_callback(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

    

In [6]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callback()

except Exception as e:
    raise e

[2022-09-30 10:40:58,992:INFO:common]: yaml file: configs\config.yaml loaded successfully
[2022-09-30 10:40:59,001:INFO:common]: yaml file: params.yaml loaded successfully
[2022-09-30 10:40:59,003:INFO:common]: created directory at: artifacts
[2022-09-30 10:40:59,005:INFO:common]: created directory at: artifacts\prepare_callbacks\checkpoint_dir
[2022-09-30 10:40:59,007:INFO:common]: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir


In [7]:
import os
os.path.dirname("x/y/t.txt")

'x/y'

In [8]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"log_{timestamp}"

'log_2022-09-30-10-40-59'